In [1]:
#%matplotlib widget

In [2]:
import pandas as pd
import os
import glob
import numpy as np
import ipyvuetify as v
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, VBox, HBox
from ipyleaflet import AwesomeIcon, Marker, Map, Icon, Popup, basemaps, Polyline
from bqplot import Axis, Figure, Lines, LinearScale
import bqplot as bq
import matplotlib.pyplot as plt
import matplotlib as mpl
import io

In [3]:
# TO DO : add title, image (inav, x-lite, moi qui vole sur l'aile, logo GAJUFPV)
# sort value before fitting ? it will destroy time slider interaction ?
# add HD video visualisation
# bug change file and time slider

In [4]:
widgets.HTML('<center> <img src="images/gaju_fpv.png" align="middle" class="center"> </center>')

HTML(value='<center> <img src="images/gaju_fpv.png" align="middle" class="center"> </center>')

In [5]:
def get_rssi(RSSI, dist):
    # sort value before fitting ?
    
    # fit data to curve
    [a,b] = np.polyfit(np.log(dist), RSSI, 1)
    rssi = a * np.log(dist) + b
    return rssi, a, b

def plot_RSSI_dist(data, data_fit, ax):
    RSSI = data['RSSI(dB)']
    dist = data['Dist(m)']
    ax.plot(dist, RSSI, '.')
    line, = ax.plot(dist[dist != 0], data_fit[0], 'r')
    ax.set_xlabel('distance (m)'), ax.set_ylabel('RSSI')
    ax.set_title('${:.2f}\ ln(x) + {:.2f}$'.format(data_fit[1],data_fit[2]))
    return line

def plot_RSSI_time(data, data_fit, ax):
    RSSI = data['RSSI(dB)']
    dist = data['Dist(m)']
    ax.plot(RSSI, '.')
    line, = ax.plot(data_fit[0], 'k')
    ax.set_xlabel('time (s)'), ax.set_ylabel('RSSI')
    return line

def get_max(data, data_type='str'):
    max_dist  = np.max(data['Dist(m)'])
    max_speed = np.max(data['GSpd(kmh)'])
    max_alt   = np.max(data['Alt(m)'])
    if data_type=="str":
        max_dist  = str(max_dist)
        max_speed = str(max_speed)
        max_alt   = str(max_alt)
    return max_dist, max_speed, max_alt

In [6]:
files = glob.glob('logs/*.csv')

In [7]:
# Get max stats
max_dist_tot, max_speed_tot, max_alt_tot = 0, 0, 0
for file in files:
    data = pd.read_csv(file, sep=';')
    new_dist, new_speed, new_alt = get_max(data, data_type='int')
    max_dist_tot  = np.max([max_dist_tot,new_dist])
    max_speed_tot = np.max([max_speed_tot,new_speed])
    max_alt_tot   = np.max([max_alt_tot,new_alt])
max_dist_tot  = str(max_dist_tot)
max_speed_tot = str(max_speed_tot)
max_alt_tot   = str(max_alt_tot)

In [8]:
class TelemetryData:
    def __init__(self):
        self.dist, self.rssi = [], []
        self.lon, self.lat   = [], []
        self.time, self.rssi_fit = [], []
        self.max_dist, self.max_speed, self.max_alt = 0, 0, 0
    def get_data(self, file):
        data = pd.read_csv(file, sep=';')
        data = data.interpolate(method='nearest', column=['GPS(lat)','GPS(lon)']).ffill().bfill()
        dist, rssi = data['Dist(m)'], data['RSSI(dB)']
        filt = (rssi != 0) & (dist != 0)
        self.dist = dist[filt] 
        self.rssi = rssi[filt]
        self.lon  = data['GPS(lon)'][filt]
        self.lat  = data['GPS(lat)'][filt]
        self.time = np.arange(len(self.rssi))
        self.rssi_fit, a, b = get_rssi(self.rssi, self.dist)
        self.max_dist, self.max_speed, self.max_alt = get_max(data)

In [9]:
# Initialize Data
tlm = TelemetryData()
tlm.get_data(files[0])

In [10]:
# MAIN COMPONENTS

In [11]:
out = widgets.Output()

In [12]:
# left - files
file_select = widgets.Select(options=[os.path.basename(f) for f in files], layout=Layout(flex='1 1 0%', width='14vw', height='50vh'))
left = VBox(children=[file_select])

# center - map and time slider
center_map = np.mean(tlm.lat), np.mean(tlm.lon)
my_map     = Map(basemap=basemaps.Esri.WorldImagery, center=center_map, zoom=15)
line_map   = Polyline(locations=[(x, y) for x, y in zip(tlm.lat, tlm.lon)], color = "red", fill=False, weight=1)
my_map.add_layer(line_map)
time_slider = widgets.IntSlider(description='Time', value=len(tlm.time), max=len(tlm.time), layout=Layout(align_content='center', justify_content="center", width='auto'))
#time_slider = v.Slider(thumb_label=True, v_model=25, description='Time', value=len(data), max=len(data), layout=Layout(align_content='center', justify_content="center", width='auto'))
center = VBox(children=[my_map, time_slider], layout=Layout(flex='1.5 1 0%', width='auto', height='80vh', align_content='center', justify_content="center"))
    
# right - rssi graphs and records
xs, ys = bq.LinearScale(), bq.LinearScale()
yax    = bq.Axis(scale=ys, label='RSSI', grid_lines='solid', orientation='vertical' )
axis   = bq.Axis(scale=xs, label='Distance (m)', grid_lines='solid')
points = bq.Scatter(x=tlm.dist, y=tlm.rssi, scales={'x':xs,'y':ys}, default_size=20)
line   = bq.Lines(x=tlm.dist, y=tlm.rssi_fit, scales={'x': xs, 'y': ys}, colors=['red'])
fig    = bq.Figure(marks=[points, line], axes=[axis, yax], layout=Layout(width='30vw', height = '45vh'))
graph_button = widgets.ToggleButtons(options=['Time', 'Distance'], description='RSSI graph', value='Distance', disabled=False, button_style='')
dist_max  = widgets.HTML(value="<b>Distance (m)</b> : "+tlm.max_dist+" / "+max_dist_tot)
speed_max = widgets.HTML(value="<b>Speed (km/h)</b> : "+tlm.max_speed+" / "+max_speed_tot)
alt_max   = widgets.HTML(value="<b>Altitude (m)</b> : "+tlm.max_alt+" / "+max_alt_tot)
trophee = widgets.HTML('<center> <img src="images/trophy.jpg" align="middle" class="center"> </center>')
records = VBox(children=[dist_max, speed_max, alt_max], layout=Layout(flex='1 1 0%', align_items='center', border='solid 1px black'))
right   = VBox(children=[graph_button, fig, trophee, records], layout=Layout(flex='1 1 0%', width='auto', height='80vh', align_items='center', flex_direction="column"))

In [13]:
# INTERACTION

In [14]:
@out.capture()
def file_change(change):
    with out:
        tlm.get_data('logs/'+change['new'])
        time_slider.max = len(tlm.time)
        my_map.center   = np.mean(tlm.lat), np.mean(tlm.lon)
        my_map.remove_layer(my_map.layers[1])
        line_map = Polyline(locations=[(x, y) for x, y in zip(tlm.lat, tlm.lon)], color = "red", fill=False, weight=1)
        my_map.add_layer(line_map)
        points.x, points.y = tlm.dist, tlm.rssi
        line.x, line.y     = tlm.dist, tlm.rssi_fit
        if graph_button.value == "Time":
            points.x, line.x = tlm.time, tlm.time
        elif graph_button.value == "Distance":
            points.x, line.x = tlm.dist, tlm.dist
        points.y, line.y = tlm.rssi, tlm.rssi_fit
        dist_max.value  = "<b>Distance (m)</b> : "+tlm.max_dist+" / "+max_dist_tot
        speed_max.value = "<b>Speed (km/h)</b> : "+tlm.max_speed+" / "+max_speed_tot
        alt_max.value   = "<b>Altitude (m)</b> : "+tlm.max_alt+" / "+max_alt_tot
file_select.observe(file_change, names='value')

# when new file, global variable should be updated !
# ==> use class ?

@out.capture()
def graph_change(change):
    with out:
        if change['new'] == 'Time':
            points.x, line.x = tlm.time, tlm.time
            axis.label = 'Time (s)'
        elif change['new'] == 'Distance':
            points.x, line.x = tlm.dist, tlm.dist
            axis.label = 'Distance (m)'
graph_button.observe(graph_change, names='value')

@out.capture()
def time_change(change):
    with out:
        last_time = change['new']
        my_map.layers[1].locations = [(x, y) for x, y in zip(tlm.lat[:last_time], tlm.lon[:last_time])]
        if graph_button.value == "Time":
            points.x, line.x = tlm.time[:last_time], tlm.time[:last_time]
        elif graph_button.value == "Distance":
            points.x, line.x = tlm.dist[:last_time], tlm.dist[:last_time]
        points.y, line.y = tlm.rssi[:last_time], tlm.rssi_fit[:last_time]
time_slider.observe(time_change, names='value')

In [15]:
# DISPLAY

In [16]:
display(HBox(children=[left, center, right]), out)

Output()

In [17]:
# upload

In [18]:
uploader = widgets.FileUpload(accept='.csv', multiple=True)
uploader

FileUpload(value={}, accept='.csv', description='Upload', multiple=True)

In [43]:
uploaded_file = uploader.value
for item in uploaded_file.keys():
    b = uploaded_file[item]['content']
    new_file = pd.read_csv(io.BytesIO(b))

In [42]:
uploaded_file.keys()

dict_keys(['AR_Wing-2020-08-26.csv'])